In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import sophius.utils as utils
import sophius.dataload as dload
from sophius.modelgen import ConvModelGenerator
from sophius.train import train_express_gpu
import torchvision.datasets as dset
import torchvision.transforms as T
from calflops import calculate_flops


VAL_SIZE = 1024

cifar10 = dset.CIFAR10('../data/CIFAR10', train=True, download=True,
                           transform=T.ToTensor())
cifar_gpu = dload.cifar_to_gpu(cifar10)

C:\Users\alexander.korolyov\miniforge3\envs\sophius\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Files already downloaded and verified


In [3]:
model_gen = ConvModelGenerator((3, 32, 32), 10, conv_num=6, lin_num=3)
model_tmpl = model_gen.generate_model_tmpl()
print(model_tmpl)
fixed_model_gpu = model_tmpl.instantiate_model().type(torch.cuda.FloatTensor)

Conv2d       (16, 32, 32)   (7, 7)   (1, 1)  
AvgPool2d    (16, 15, 15)   (4, 4)   (2, 2)  
LeakyReLU    (16, 15, 15)   (0.1)   
BatchNorm2d  (16, 15, 15)  
Conv2d       (8, 4, 4)      (4, 4)   (3, 3)  
BatchNorm2d  (8, 4, 4)     
LeakyReLU    (8, 4, 4)      (0.01)  
GlobalAvgPool2d (8, 1, 1)     
Flatten      8             
Linear       10            



In [7]:
flops, macs, params = calculate_flops(model=fixed_model_gpu, 
                                      input_shape=(1, 3, 32, 32),
                                      output_as_string=False,
                                      print_results=False,
                                      print_detailed=False,
                                      output_precision=4)
print("FLOPs:%s   MACs:%s   Params:%s \n" %(flops, macs, params))

FLOPs:9853600   MACs:4882592   Params:4562 



In [10]:
utils.calc_model_flops(fixed_model_gpu, (3, 32, 32))

9853600

In [2]:
import pandas as pd
import seaborn as sns
from tqdm import tqdm

loader_gpu = dload.get_loader_gpu(cifar_gpu, val_size=VAL_SIZE, batch_size=1024)

df = pd.DataFrame(columns=['flops', 'macs', 'params', 'time', 'val_acc', 'train_acc'])

for i in tqdm(range(100)):
    model = model_gen.generate_model().type(torch.cuda.FloatTensor)
    flops, macs, params = calculate_flops(model=model, 
                                      input_shape=(1, 3, 32, 32),
                                      output_as_string=False,
                                      print_results=False,
                                      print_detailed=False,
                                      output_precision=4)
    # express train
    t, val_acc, train_acc = train_express_gpu(model = model,
                                              train = True,
                                              loader = loader_gpu,
                                              milestones = [],
                                              num_epoch = 10,
                                              verbose = False)
    
    df.loc[i] = [flops, macs, params, t, val_acc, train_acc]
    # print(df.loc[i])

NameError: name 'dload' is not defined

In [ ]:
import matplotlib.pyplot as plt

sns.scatterplot(df, x='time', y='flops');
plt.yscale('log')
plt.xscale('log')